In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import plotly.express as px
import scipy
from skimage import color, exposure, measure, morphology, util
from segmentflow import segment, view
%load_ext autoreload
%autoreload 2

## Load images

In [ ]:
# imgs_path = Path('../data/F83_01')
imgs_path = Path('../data/F83_01_med')
slices = [250, 500, 750]
# imgs = segment.load_images(
imgs_med = segment.load_images(
    imgs_path,
    slice_crop=None,
    # convert_to_float=True,
    file_suffix='.tif'
)
fig, axes = view.plot_slices(
    # imgs,
    imgs_med,
    slices=slices,
    print_slices=True,
    imgs_per_row=None,
    fig_w=7.5,
    dpi=100
)

In [ ]:
# imgs_med = segment.preprocess(imgs, median_filter=True)

In [ ]:
med_hist, med_bins_centers = exposure.histogram(imgs_med)
fig, ax = plt.subplots()
ax.plot(med_bins_centers, med_hist)

In [ ]:
lo = np.percentile(imgs_med, 25.5)
hi = np.percentile(imgs_med, 99.98)
fig, ax = plt.subplots()
ax.plot(med_bins_centers, med_hist)
ax.axvline(lo, c='red')
ax.axvline(hi, c='red')
plt.show()

## Rescale intensity

In [ ]:
imgs = None
imgs_res = None
imgs_med = segment.preprocess(imgs_med, rescale_intensity_range=[25.5, 99.98])
fig, axes = view.plot_slices(
    imgs_med,
    slices=slices,
    print_slices=True,
    imgs_per_row=None,
    fig_w=7.5,
    dpi=100
)

In [ ]:
segment.save_images(imgs_med, Path('../data/F83_01_pre'))

In [ ]:
hist, bins_centers = exposure.histogram(imgs_res)
fig, ax = plt.subplots()
ax.plot(bins_centers, hist)
ax.set_ylim([0, 1e5])
plt.show()

In [ ]:
thresholds, fig, ax = segment.threshold_multi_min(
    imgs_res, nbins=150, nthresholds='all', return_fig_ax=True, ylims=[0, 2e7])
plt.show()

In [ ]:
imgs_semantic = segment.isolate_classes(imgs_med, thresholds)
for i in range(len(thresholds) + 1):
    print('imgs_semantic =', i)
    fig, axes = view.plot_slices(
        imgs_semantic==i,
        slices=slices,
        print_slices=False,
        fig_w=7.5,
        dpi=100
    )
    plt.show()

In [ ]:
imgs = None
imgs_med = None
imgs_res = None
imgs_labeled = segment.watershed_segment(
    imgs_semantic==len(thresholds),
    min_peak_distance=6,
    exclude_borders=True,
    return_dict=False
)
fig, axes = view.plot_color_labels(
    imgs_labeled,
    slices=slices,
    fig_w=7.5,
    dpi=100
)